In [1]:
!pip install -q scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results'
feature_selection_results = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'
sys.path.insert(0,code_path)
from oselm import set_use_know
from SharedFunctions import main,generate_oselm_models

In [5]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))

In [6]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']

In [7]:
# # use_know = True
# # set_use_know(use_know)
# # path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
# # os.makedirs(path, exist_ok=True)
# # for f_name,d_name in zip(filenames,data_name):
# #   DFS_results_path = os.path.join(feature_selection_results,d_name)
# #   main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
# #       feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,DFS_results_path=DFS_results_path)

In [8]:
DATA_NUMBER=0
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : kddcup99_csv.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.9955099422706863, 'precision': 1.0, 'recall': 0.991494532199271, 'f1-score': 0.9957291031116534, 'auc': 0.9957472660996355}


  0%|          | 0/7832 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.9639240506329114, 'precision': 0.9379391100702577, 'recall': 0.9950310559006211, 'f1-score': 0.9656419529837251, 'auc': 0.9633219795632137}


  0%|          | 0/7892 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.9879670677644079, 'precision': 0.9864864864864865, 'recall': 0.9901356350184957, 'f1-score': 0.9883076923076923, 'auc': 0.9879063591759144}


  0%|          | 0/7790 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.9922978177150192, 'precision': 0.9897172236503856, 'recall': 0.9948320413436692, 'f1-score': 0.9922680412371134, 'auc': 0.992313979855508}


In [9]:
DATA_NUMBER=1
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : ISCX2012.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.6339341205717837, 'precision': 0.5692913385826772, 'recall': 0.9450980392156862, 'f1-score': 0.7105651105651106, 'auc': 0.6484968869064214}


  0%|          | 0/8018 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.7066167290886392, 'precision': 0.993920972644377, 'recall': 0.41132075471698115, 'f1-score': 0.5818505338078293, 'auc': 0.7044212199855042}


  0%|          | 0/8138 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.6363636363636364, 'precision': 0.9872340425531915, 'recall': 0.28258221680876977, 'f1-score': 0.4393939393939394, 'auc': 0.6394323723449052}


  0%|          | 0/8122 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.5076923076923077, 'precision': 0.90625, 'recall': 0.035108958837772396, 'f1-score': 0.06759906759906759, 'auc': 0.515677132735532}


In [ ]:
DATA_NUMBER=2
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CSE-CIC2018.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 0.5}


  0%|          | 0/8000 [00:00<?, ?it/s]

...

  0%|          | 0/15984 [00:00<?, ?it/s]

DFS
7 {'accuracy': 0.8529871754770098, 'precision': 0.7710667316122747, 'recall': 1.0, 'f1-score': 0.8707370737073707, 'auc': 0.8543990086741016}


  0%|          | 0/15978 [00:00<?, ?it/s]

DFS
8 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 1.0}


  0%|          | 0/15962 [00:00<?, ?it/s]

In [ ]:
DATA_NUMBER=3
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=4
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=5
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=6
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=7
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=8
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)